# Crime Trends Analysis Tool v2.0

The Crime Trends Analysis Tool analyzes citywide crime trends and surfaces problematic crime conditions, enabling commanders to begin problem-solving immediately.

In [1]:
import datetime as dt
import os.path
import urllib.request
from dateutil.relativedelta import relativedelta
import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

## Loading Data & Cleaning
For the purposes of this demo, you can download NYPD's open data below.

### Example with NYPD Open Data

In [3]:
# Download CSV files with historic crime data and save to current directory 
# Note: this may take 10+ minutes based on connection speed

if not os.path.isfile('NYPD-Complaint-Date_Current-YTD.csv'):
    print ('Downloading: NYPD-Complaint-Date_Current-YTD.csv') # size ~60MB
    urllib.request.urlretrieve('https://data.cityofnewyork.us/api/views/5uac-w243/rows.csv?accessType=DOWNLOAD', 
                               'NYPD-Complaint-Date_Current-YTD.csv')

if not os.path.isfile('NYPD-Complaint-Data-Historic.csv'):
    print ('Downloading: NYPD-Complaint-Data-Historic.csv') #size ~1.43GB
    urllib.request.urlretrieve('https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD', 
                               'NYPD-Complaint-Data-Historic.csv')

In [4]:
# Combine the historical complaint data with the current YTD complaint data
def get_crime_data():
    dfs = []
    use_cols = ['CMPLNT_NUM', 'RPT_DT', 'ADDR_PCT_CD', 'OFNS_DESC']
    files = ['NYPD-Complaint-Date_Current-YTD.csv', 'NYPD-Complaint-Data-Historic.csv']
    for fn in files:
        print ('Reading data from:', fn)
        df = pd.read_csv(fn, usecols=use_cols)
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True, axis=0)
     
crimes = get_crime_data()

def clean_crime_data(crimes):
    # Data cleaning, reducing memory usage
    crimes['RPT_DT'] = pd.to_datetime(crimes['RPT_DT'], format='%m/%d/%Y', errors='coerce')
    crimes['OFNS_DESC'] = crimes['OFNS_DESC'].astype('category')
    crimes.dropna(inplace=True)
    crimes['ADDR_PCT_CD'] = pd.to_numeric(crimes['ADDR_PCT_CD'], downcast='unsigned')
    crimes = crimes[['CMPLNT_NUM', 'RPT_DT', 'ADDR_PCT_CD', 'OFNS_DESC']]
    crimes.columns = ['id', 'date', 'pct', 'crime']
    
    return crimes

crimes = clean_crime_data(crimes)

Reading data from: NYPD-Complaint-Date_Current-YTD.csv
Reading data from: NYPD-Complaint-Data-Historic.csv


In [5]:
crimes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5788105 entries, 0 to 5807405
Data columns (total 4 columns):
id       int64
date     datetime64[ns]
pct      uint8
crime    category
dtypes: category(1), datetime64[ns](1), int64(1), uint8(1)
memory usage: 143.5 MB


In [6]:
crimes.head()

,id,date,pct,crime
0,417734073,2017-06-30,90,CRIMINAL MISCHIEF & RELATED OF
1,433995840,2017-06-30,32,HARRASSMENT 2
2,641950009,2017-06-30,73,DANGEROUS DRUGS
3,220585856,2017-06-30,9,ROBBERY
4,272762078,2017-06-30,47,HARRASSMENT 2


### User-defined Constants

* **`YEARS_TO_ANALYZE`**: Number of years to generate trend data for (default: 1 year)
* **`REPORT_DATE`**: Ending date to generate trend data (note: must be a Sunday to align with NYPD reporting periods)

In [7]:
# Get the first January 1 date for which the dataset includes a full year of data
FIRST_CAL_YEAR_DT = crimes['date'].min().to_pydatetime()
if (FIRST_CAL_YEAR_DT.month != 1) and (FIRST_CAL_YEAR_DT.day != 1):
    FIRST_CAL_YEAR_DT = dt.datetime(min_date.year + 1, 1, 1)

# For this purposes of this report, the weekly crime reporting period is Monday - Sunday. So,
# we need to find the last Sunday in the dataset.
max_date = crimes['date'].max().to_pydatetime()
idx = (max_date.weekday() + 1) % 7
REPORT_END_DT = max_date - dt.timedelta(7+idx)

# Number of years of historical trends to generate, change as desired (recommend 0 or 1)
YEARS_TO_ANALYZE = 5

REPORT_START_DT = REPORT_END_DT - relativedelta(weeks=52 * YEARS_TO_ANALYZE) - relativedelta(weeks=55, days=6)

DATA_START_DT = dt.datetime(REPORT_START_DT.year, 1, 1) # necessary for calculating YTD counts

In [8]:
print('First Full Calendar Year Date:', FIRST_CAL_YEAR_DT.strftime('%Y-%m-%d'))
print('Report Start Date:', REPORT_START_DT.strftime('%Y-%m-%d'))
print('Report End Date:', REPORT_END_DT.strftime('%Y-%m-%d'))
print('Data Start Date:', DATA_START_DT.strftime('%Y-%m-%d'))

First Full Calendar Year Date: 2006-01-01
Report Start Date: 2011-05-30
Report End Date: 2017-06-18
Data Start Date: 2011-01-01


### Generate trends data

In [9]:
# Filter crime data to only include incidents between the DATA_START_DT and REPORT_END_DT, and
# generate daily counts for each crime type by precinct

def create_daily_counts(crimes):
    crimes = crimes.set_index('date').sort_index()
    crimes = crimes[DATA_START_DT:REPORT_END_DT]
    
    # Create daily counts of crime types in every precinct, starting with earliest day for which
    # there is data
    incidents_daily = crimes.groupby(['pct', 'crime'])['id'].resample('D').count()
    
    # Because not every crime occurs at least once every day in each precinct, the daily counts generated above
    # start on the date of the first time a particular crime type occurred in the dataset. 
    level_0 = incidents_daily.index.get_level_values('pct').unique()
    level_1 = incidents_daily.index.get_level_values('crime').unique()
    level_2 = pd.date_range(REPORT_START_DT, REPORT_END_DT)
    multi_index = pd.MultiIndex.from_product([level_0, level_1, level_2],
                                             names=['pct', 'crime', 'date'])

    incidents_daily = incidents_daily.reindex(multi_index).fillna(0)
    incidents_daily.name = 'daily_count'
    
    incidents_daily = pd.to_numeric(incidents_daily, downcast='unsigned')
    
    return incidents_daily

incidents_daily = create_daily_counts(crimes)

incidents_daily.head()

pct  crime                date      
1    ADMINISTRATIVE CODE  2011-05-30    0
                          2011-05-31    0
                          2011-06-01    0
                          2011-06-02    0
                          2011-06-03    0
Name: daily_count, dtype: uint8

In [10]:
# Create weekly and 28-day crime counts for each crime type by precinct, in addition
# to percent change compared to the same period one year ago

def create_7_and_28_day_counts(incidents_daily):

    # Create weekly counts of incidents (note: date label on ending Sunday)
    incidents_7_day = (incidents_daily.unstack(level=[0, 1]) 
                                      .resample('7D', loffset='6D')
                                      .sum()
                                      .stack(level=[1, 0])
                                      .swaplevel(2,0))
    incidents_7_day.name = 'incidents_7_day'

    # Create 28-day counts of incidents
    incidents_28_day = (incidents_7_day.groupby(level=['pct', 'crime'])
                                       .rolling(window=4)
                                       .sum()
                                       .dropna()
                                       .astype(int))
    incidents_28_day.reset_index(level=[0, 1], drop=True, inplace=True)
    incidents_28_day.name = 'incidents_28_day'

    incidents = pd.concat([incidents_7_day, incidents_28_day], 
                          axis=1, join_axes=[incidents_28_day.index])
    
    # cleanup - there appears to be some very high 28-day counts that don't seem possible
    incidents.loc[(incidents['incidents_28_day'] > 150), 'incidents_28_day'] = np.nan
    
    # create prior year counts
    group = incidents.groupby(level=['pct', 'crime'])
    incidents['incidents_28_day_prior_yr'] = group['incidents_28_day'].shift(52).values
    incidents['incidents_7_day_prior_yr'] = group['incidents_7_day'].shift(52).values
        
    # Calculate percent changes
    incidents['7_day_pct_change'] = (((incidents['incidents_7_day'] - incidents['incidents_7_day_prior_yr'])
                                     / incidents['incidents_7_day_prior_yr']))
    incidents['7_day_pct_change'] = incidents['7_day_pct_change'].round(4).fillna('*.*').replace(np.inf, '*.*')

    incidents['28_day_pct_change'] = (((incidents['incidents_28_day'] - incidents['incidents_28_day_prior_yr'])
                                     / incidents['incidents_28_day_prior_yr']))
    incidents['28_day_pct_change'] = incidents['28_day_pct_change'].round(4).fillna('*.*').replace(np.inf, '*.*')
    
    return incidents

incidents = create_7_and_28_day_counts(incidents_daily)

incidents.head()

incidents_7_day  incidents_28_day  \
pct crime               date                                            
1   ADMINISTRATIVE CODE 2011-06-26                0               0.0   
                        2011-07-03                0               0.0   
                        2011-07-10                0               0.0   
                        2011-07-17                0               0.0   
                        2011-07-24                0               0.0   

                                    incidents_28_day_prior_yr  \
pct crime               date                                    
1   ADMINISTRATIVE CODE 2011-06-26                        NaN   
                        2011-07-03                        NaN   
                        2011-07-10                        NaN   
                        2011-07-17                        NaN   
                        2011-07-24                        NaN   

                                    incidents_7_day_prior_yr 7_day_pct_change  \
pct crime               date                                                    
1   ADMINISTRATIVE CODE 2011-06-26                       NaN              *.*   
                        2011-07-03                       NaN              *.*   
                        2011-07-10                       NaN              *.*   
                        2011-07-17                       NaN              *.*   
                        2011-07-24                       NaN              *.*   

                                   28_day_pct_change  
pct crime               date                          
1   ADMINISTRATIVE CODE 2011-06-26               *.*  
                        2011-07-03               *.*  
                        2011-07-10               *.*  
                        2011-07-17               *.*  
                        2011-07-24               *.*

In [11]:
# Calculate rolling 12, 26, and 52 week averages and standard deviations

def create_rolling_avg_std(incidents, windows=[12, 26, 52]):
    group = incidents.groupby(level=['pct', 'crime'])
    for window in windows:
        incidents[str(window) + '_week_avg'] = group['incidents_28_day'].rolling(window=window).mean().values
        incidents[str(window) + '_week_std'] = group['incidents_28_day'].rolling(window=window).std().values
    return incidents

incidents = create_rolling_avg_std(incidents)

incidents.head()

incidents_7_day  incidents_28_day  \
pct crime               date                                            
1   ADMINISTRATIVE CODE 2011-06-26                0               0.0   
                        2011-07-03                0               0.0   
                        2011-07-10                0               0.0   
                        2011-07-17                0               0.0   
                        2011-07-24                0               0.0   

                                    incidents_28_day_prior_yr  \
pct crime               date                                    
1   ADMINISTRATIVE CODE 2011-06-26                        NaN   
                        2011-07-03                        NaN   
                        2011-07-10                        NaN   
                        2011-07-17                        NaN   
                        2011-07-24                        NaN   

                                    incidents_7_day_prior_yr 7_day_pct_change  \
pct crime               date                                                    
1   ADMINISTRATIVE CODE 2011-06-26                       NaN              *.*   
                        2011-07-03                       NaN              *.*   
                        2011-07-10                       NaN              *.*   
                        2011-07-17                       NaN              *.*   
                        2011-07-24                       NaN              *.*   

                                   28_day_pct_change  12_week_avg  \
pct crime               date                                        
1   ADMINISTRATIVE CODE 2011-06-26               *.*          NaN   
                        2011-07-03               *.*          NaN   
                        2011-07-10               *.*          NaN   
                        2011-07-17               *.*          NaN   
                        2011-07-24               *.*          NaN   

                                    12_week_std  26_week_avg  26_week_std  \
pct crime               date                                                
1   ADMINISTRATIVE CODE 2011-06-26          NaN          NaN          NaN   
                        2011-07-03          NaN          NaN          NaN   
                        2011-07-10          NaN          NaN          NaN   
                        2011-07-17          NaN          NaN          NaN   
                        2011-07-24          NaN          NaN          NaN   

                                    52_week_avg  52_week_std  
pct crime               date                                  
1   ADMINISTRATIVE CODE 2011-06-26          NaN          NaN  
                        2011-07-03          NaN          NaN  
                        2011-07-10          NaN          NaN  
                        2011-07-17          NaN          NaN  
                        2011-07-24          NaN          NaN

In [12]:
# This function finds the mean and standard deviation of the same 28 day period over the number 
# of years available in the dataset (e.g. week 4 in 2017 vs week 4 in 2016 vs week 4 in 2015).
# A minimum of 3 years is recommended. This helps account for seasonality and holidays.

def create_avg_std_of_same_28_day_period(incidents):
    df = incidents['incidents_28_day'].reset_index()
    df['week'] = df['date'].dt.week
    group = df.groupby(['pct', 'crime', 'week'])['incidents_28_day']
    avg = group.mean()
    avg.name = 'same_28_day_avg'
    std = group.std()
    std.name = 'same_28_day_std'
    week_comp = pd.concat([avg, std], axis=1).fillna(0).reset_index()
    last_yr = df.groupby(['pct', 'crime']).tail(53)
    temp = pd.merge(last_yr[['pct', 'crime', 'week', 'date']], 
                    week_comp, on=['pct', 'crime', 'week']).drop('week', axis=1)
    temp = temp.set_index(['pct', 'crime', 'date']).sort_index(level=['pct', 'crime', 'date'])
    incidents = pd.concat([incidents, temp], axis=1)
    return incidents

incidents = create_avg_std_of_same_28_day_period(incidents)

incidents.head()

incidents_7_day  incidents_28_day  \
pct crime    date                                            
1   ABORTION 2011-06-26                0               0.0   
             2011-07-03                0               0.0   
             2011-07-10                0               0.0   
             2011-07-17                0               0.0   
             2011-07-24                0               0.0   

                         incidents_28_day_prior_yr  incidents_7_day_prior_yr  \
pct crime    date                                                              
1   ABORTION 2011-06-26                        NaN                       NaN   
             2011-07-03                        NaN                       NaN   
             2011-07-10                        NaN                       NaN   
             2011-07-17                        NaN                       NaN   
             2011-07-24                        NaN                       NaN   

                        7_day_pct_change 28_day_pct_change  12_week_avg  \
pct crime    date                                                         
1   ABORTION 2011-06-26              *.*               *.*          NaN   
             2011-07-03              *.*               *.*          NaN   
             2011-07-10              *.*               *.*          NaN   
             2011-07-17              *.*               *.*          NaN   
             2011-07-24              *.*               *.*          NaN   

                         12_week_std  26_week_avg  26_week_std  52_week_avg  \
pct crime    date                                                             
1   ABORTION 2011-06-26          NaN          NaN          NaN          NaN   
             2011-07-03          NaN          NaN          NaN          NaN   
             2011-07-10          NaN          NaN          NaN          NaN   
             2011-07-17          NaN          NaN          NaN          NaN   
             2011-07-24          NaN          NaN          NaN          NaN   

                         52_week_std  same_28_day_avg  same_28_day_std  
pct crime    date                                                       
1   ABORTION 2011-06-26          NaN              NaN              NaN  
             2011-07-03          NaN              NaN              NaN  
             2011-07-10          NaN              NaN              NaN  
             2011-07-17          NaN              NaN              NaN  
             2011-07-24          NaN              NaN              NaN

In [13]:
# Creates a flag if crime is increasing or decreasing four weeks in a row.
# This could be used as an alert trigger.

def create_four_conseq_inc_dec_flg(incidents):
    temp = incidents['incidents_28_day'].to_frame()
    group = temp.groupby(level=['pct', 'crime'])['incidents_28_day']
    
    temp['prev'] = group.shift(1)
    temp['prev_prev'] = group.shift(2)
    temp['prev_prev_prev'] = group.shift(3)
    
    four_increase = ((temp['incidents_28_day'] > temp['prev']) &  
                     (temp['prev'] > temp['prev_prev']) &
                     (temp['prev_prev'] > temp['prev_prev_prev']))

    four_decrease = ((temp['incidents_28_day'] < temp['prev']) &  
                     (temp['prev'] < temp['prev_prev']) &
                     (temp['prev_prev'] < temp['prev_prev_prev']))
    
    incidents['four_conseq_inc'] = four_increase
    incidents['four_conseq_dec'] = four_decrease

    return incidents

incidents = create_four_conseq_inc_dec_flg(incidents)

incidents.head()

incidents_7_day  incidents_28_day  \
pct crime    date                                            
1   ABORTION 2011-06-26                0               0.0   
             2011-07-03                0               0.0   
             2011-07-10                0               0.0   
             2011-07-17                0               0.0   
             2011-07-24                0               0.0   

                         incidents_28_day_prior_yr  incidents_7_day_prior_yr  \
pct crime    date                                                              
1   ABORTION 2011-06-26                        NaN                       NaN   
             2011-07-03                        NaN                       NaN   
             2011-07-10                        NaN                       NaN   
             2011-07-17                        NaN                       NaN   
             2011-07-24                        NaN                       NaN   

                        7_day_pct_change 28_day_pct_change  12_week_avg  \
pct crime    date                                                         
1   ABORTION 2011-06-26              *.*               *.*          NaN   
             2011-07-03              *.*               *.*          NaN   
             2011-07-10              *.*               *.*          NaN   
             2011-07-17              *.*               *.*          NaN   
             2011-07-24              *.*               *.*          NaN   

                         12_week_std  26_week_avg  26_week_std  52_week_avg  \
pct crime    date                                                             
1   ABORTION 2011-06-26          NaN          NaN          NaN          NaN   
             2011-07-03          NaN          NaN          NaN          NaN   
             2011-07-10          NaN          NaN          NaN          NaN   
             2011-07-17          NaN          NaN          NaN          NaN   
             2011-07-24          NaN          NaN          NaN          NaN   

                         52_week_std  same_28_day_avg  same_28_day_std  \
pct crime    date                                                        
1   ABORTION 2011-06-26          NaN              NaN              NaN   
             2011-07-03          NaN              NaN              NaN   
             2011-07-10          NaN              NaN              NaN   
             2011-07-17          NaN              NaN              NaN   
             2011-07-24          NaN              NaN              NaN   

                         four_conseq_inc  four_conseq_dec  
pct crime    date                                          
1   ABORTION 2011-06-26            False            False  
             2011-07-03            False            False  
             2011-07-10            False            False  
             2011-07-17            False            False  
             2011-07-24            False            False

In [14]:
# Some clean up

def clean_up(incidents):
    # Because we shifted 52 weeks, we drop all weeks that now have null values
    incidents.dropna(inplace=True)

    # Convert to integers from floats
    incidents['incidents_28_day_prior_yr'] = pd.to_numeric(incidents['incidents_28_day_prior_yr'],
                                                           downcast='unsigned')

    incidents['incidents_7_day_prior_yr'] = pd.to_numeric(incidents['incidents_7_day_prior_yr'],
                                                          downcast='unsigned')

    return incidents

incidents = clean_up(incidents)

incidents.head()

incidents_7_day  incidents_28_day  \
pct crime    date                                            
1   ABORTION 2016-06-19                0               0.0   
             2016-06-26                0               0.0   
             2016-07-03                0               0.0   
             2016-07-10                0               0.0   
             2016-07-17                0               0.0   

                         incidents_28_day_prior_yr  incidents_7_day_prior_yr  \
pct crime    date                                                              
1   ABORTION 2016-06-19                          0                         0   
             2016-06-26                          0                         0   
             2016-07-03                          0                         0   
             2016-07-10                          0                         0   
             2016-07-17                          0                         0   

                        7_day_pct_change 28_day_pct_change  12_week_avg  \
pct crime    date                                                         
1   ABORTION 2016-06-19              *.*               *.*          0.0   
             2016-06-26              *.*               *.*          0.0   
             2016-07-03              *.*               *.*          0.0   
             2016-07-10              *.*               *.*          0.0   
             2016-07-17              *.*               *.*          0.0   

                         12_week_std  26_week_avg  26_week_std  52_week_avg  \
pct crime    date                                                             
1   ABORTION 2016-06-19          0.0          0.0          0.0          0.0   
             2016-06-26          0.0          0.0          0.0          0.0   
             2016-07-03          0.0          0.0          0.0          0.0   
             2016-07-10          0.0          0.0          0.0          0.0   
             2016-07-17          0.0          0.0          0.0          0.0   

                         52_week_std  same_28_day_avg  same_28_day_std  \
pct crime    date                                                        
1   ABORTION 2016-06-19          0.0              0.0              0.0   
             2016-06-26          0.0              0.0              0.0   
             2016-07-03          0.0              0.0              0.0   
             2016-07-10          0.0              0.0              0.0   
             2016-07-17          0.0              0.0              0.0   

                         four_conseq_inc  four_conseq_dec  
pct crime    date                                          
1   ABORTION 2016-06-19            False            False  
             2016-06-26            False            False  
             2016-07-03            False            False  
             2016-07-10            False            False  
             2016-07-17            False            False

In [15]:
# Note: this can take a long time if you have a lot of data (consider reducing the number of 
# years to analyze to decrease the size of the dataset)
# with NYPD data, 1 year takes ~8.5 minutes

# Here we create a new index to ensure every date has an entry and we will fill days with no crimes 
# with the value of zero

def create_daily_counts_from_start_date(incidents_daily):
    level_0 = incidents_daily.index.get_level_values('pct').unique()
    level_1 = incidents_daily.index.get_level_values('crime').unique()
    level_2 = pd.date_range(DATA_START_DT, REPORT_END_DT)
    multi_index = pd.MultiIndex.from_product([level_0, level_1, level_2], names=['pct', 'crime', 'date'])

    incidents_daily = incidents_daily.reindex(multi_index).fillna(0).astype(int)
    
    return incidents_daily

def ytd_count(pct, crime, date, kind='current'):
    date = date.to_pydatetime()
    if kind == 'current':
        date_start = dt.date(date.year, 1, 1).strftime('%Y-%m-%d')
        date_end = date.strftime('%Y-%m-%d')
    elif kind == 'previous':
        date_start = dt.date(date.year-1, 1, 1).strftime('%Y-%m-%d')
        date_end = (date - relativedelta(years=1)).strftime('%Y-%m-%d')
    counts = incidents_daily_strt[(pct, crime)][date_start:date_end].sum()
    return counts

def get_ytd_counts(incidents):
    incidents_temp = incidents.reset_index()[['pct', 'crime', 'date']]

    incidents['incidents_ytd'] = incidents_temp.apply(lambda row: ytd_count(row['pct'], row['crime'], row['date']), 
                                                      axis=1).values

    incidents['incidents_ytd_prior_yr'] = incidents_temp.apply(lambda row: ytd_count(row['pct'], row['crime'], 
                                                                                     row['date'],
                                                                                     kind='previous'), 
                                                               axis=1).values
    
    incidents['ytd_pct_change'] = (((incidents['incidents_ytd'] - incidents['incidents_ytd_prior_yr'])
                              / incidents['incidents_ytd_prior_yr']))
    incidents['ytd_pct_change'] = incidents['ytd_pct_change'].round(4).fillna('*.*').replace(np.inf, '*.*')
    
    return incidents

incidents_daily_strt = create_daily_counts_from_start_date(incidents_daily)

incidents = get_ytd_counts(incidents)

incidents.head()

incidents_7_day  incidents_28_day  \
pct crime    date                                            
1   ABORTION 2016-06-19                0               0.0   
             2016-06-26                0               0.0   
             2016-07-03                0               0.0   
             2016-07-10                0               0.0   
             2016-07-17                0               0.0   

                         incidents_28_day_prior_yr  incidents_7_day_prior_yr  \
pct crime    date                                                              
1   ABORTION 2016-06-19                          0                         0   
             2016-06-26                          0                         0   
             2016-07-03                          0                         0   
             2016-07-10                          0                         0   
             2016-07-17                          0                         0   

                        7_day_pct_change 28_day_pct_change  12_week_avg  \
pct crime    date                                                         
1   ABORTION 2016-06-19              *.*               *.*          0.0   
             2016-06-26              *.*               *.*          0.0   
             2016-07-03              *.*               *.*          0.0   
             2016-07-10              *.*               *.*          0.0   
             2016-07-17              *.*               *.*          0.0   

                         12_week_std  26_week_avg  26_week_std  52_week_avg  \
pct crime    date                                                             
1   ABORTION 2016-06-19          0.0          0.0          0.0          0.0   
             2016-06-26          0.0          0.0          0.0          0.0   
             2016-07-03          0.0          0.0          0.0          0.0   
             2016-07-10          0.0          0.0          0.0          0.0   
             2016-07-17          0.0          0.0          0.0          0.0   

                         52_week_std  same_28_day_avg  same_28_day_std  \
pct crime    date                                                        
1   ABORTION 2016-06-19          0.0              0.0              0.0   
             2016-06-26          0.0              0.0              0.0   
             2016-07-03          0.0              0.0              0.0   
             2016-07-10          0.0              0.0              0.0   
             2016-07-17          0.0              0.0              0.0   

                         four_conseq_inc  four_conseq_dec  incidents_ytd  \
pct crime    date                                                          
1   ABORTION 2016-06-19            False            False              0   
             2016-06-26            False            False              0   
             2016-07-03            False            False              0   
             2016-07-10            False            False              0   
             2016-07-17            False            False              0   

                         incidents_ytd_prior_yr ytd_pct_change  
pct crime    date                                               
1   ABORTION 2016-06-19                       0            *.*  
             2016-06-26                       0            *.*  
             2016-07-03                       0            *.*  
             2016-07-10                       0            *.*  
             2016-07-17                       0            *.*

**Use the "Poisson Z-Score" rather than regular Z-Score.** Justification in source below.

Source: Wheeler, A. P. (2016). Tables and graphs for monitoring temporal crime trends. International Journal of Police Science & Management, 18(3), 159-172. doi:10.1177/1461355716642781
http://journals.sagepub.com/doi/abs/10.1177/1461355716642781?journalCode=psma

In [16]:
def calc_poisson_zscore(incidents, windows=[12, 26, 52]):
    for window in windows:
        incidents[str(window)+'_week_pzscore'] = (2 * (np.sqrt(incidents['incidents_28_day'].astype('float')) - \
                                                       np.sqrt(incidents[str(window)+'_week_avg'].astype('float')))
                                                 ).round(2)
    incidents['same_28_day_pzscore'] = (2 * (np.sqrt(incidents['incidents_28_day'].astype('float')) - \
                                                       np.sqrt(incidents['same_28_day_avg'].astype('float')))
                                        ).round(2)
    return incidents

incidents = calc_poisson_zscore(incidents)

incidents.head()

incidents_7_day  incidents_28_day  \
pct crime    date                                            
1   ABORTION 2016-06-19                0               0.0   
             2016-06-26                0               0.0   
             2016-07-03                0               0.0   
             2016-07-10                0               0.0   
             2016-07-17                0               0.0   

                         incidents_28_day_prior_yr  incidents_7_day_prior_yr  \
pct crime    date                                                              
1   ABORTION 2016-06-19                          0                         0   
             2016-06-26                          0                         0   
             2016-07-03                          0                         0   
             2016-07-10                          0                         0   
             2016-07-17                          0                         0   

                        7_day_pct_change 28_day_pct_change  12_week_avg  \
pct crime    date                                                         
1   ABORTION 2016-06-19              *.*               *.*          0.0   
             2016-06-26              *.*               *.*          0.0   
             2016-07-03              *.*               *.*          0.0   
             2016-07-10              *.*               *.*          0.0   
             2016-07-17              *.*               *.*          0.0   

                         12_week_std  26_week_avg  26_week_std  52_week_avg  \
pct crime    date                                                             
1   ABORTION 2016-06-19          0.0          0.0          0.0          0.0   
             2016-06-26          0.0          0.0          0.0          0.0   
             2016-07-03          0.0          0.0          0.0          0.0   
             2016-07-10          0.0          0.0          0.0          0.0   
             2016-07-17          0.0          0.0          0.0          0.0   

                         52_week_std  same_28_day_avg  same_28_day_std  \
pct crime    date                                                        
1   ABORTION 2016-06-19          0.0              0.0              0.0   
             2016-06-26          0.0              0.0              0.0   
             2016-07-03          0.0              0.0              0.0   
             2016-07-10          0.0              0.0              0.0   
             2016-07-17          0.0              0.0              0.0   

                         four_conseq_inc  four_conseq_dec  incidents_ytd  \
pct crime    date                                                          
1   ABORTION 2016-06-19            False            False              0   
             2016-06-26            False            False              0   
             2016-07-03            False            False              0   
             2016-07-10            False            False              0   
             2016-07-17            False            False              0   

                         incidents_ytd_prior_yr ytd_pct_change  \
pct crime    date                                                
1   ABORTION 2016-06-19                       0            *.*   
             2016-06-26                       0            *.*   
             2016-07-03                       0            *.*   
             2016-07-10                       0            *.*   
             2016-07-17                       0            *.*   

                         12_week_pzscore  26_week_pzscore  52_week_pzscore  \
pct crime    date                                                            
1   ABORTION 2016-06-19              0.0              0.0              0.0   
             2016-06-26              0.0              0.0              0.0   
             2016-07-03              0.0              0.0              0.0   
             2016-07-10              0.0    

In [17]:
# Classify each 28-day period for each crime type in each precint to one of 5 trend categories based
# on the z-score (the number of standard deviations from the mean)
def classify_trend(x):
    if x >= 2:
        return 'Spike'
    elif x >= 1:
        return 'Condition'
    elif x <= -2:
        return 'Major Reduction'
    elif x <= -1:
        return 'Decrease'
    else:
        return 'Normal'
    
def create_alert_score_and_classify(incidents):
    cols = ['12_week_pzscore', '26_week_pzscore', '52_week_pzscore', 'same_28_day_pzscore']
    
    # Correct for any outliers
    for col in cols:
        incidents.loc[incidents[col] > 3, col] = 3
        incidents.loc[incidents[col] < -3, col] = -3

    incidents['alert_mean'] = incidents[cols].mean(axis=1)
    
    incidents['trend_class'] = incidents['alert_mean'].apply(classify_trend)
    
    return incidents
    
incidents = create_alert_score_and_classify(incidents)

incidents.head()

incidents_7_day  incidents_28_day  \
pct crime    date                                            
1   ABORTION 2016-06-19                0               0.0   
             2016-06-26                0               0.0   
             2016-07-03                0               0.0   
             2016-07-10                0               0.0   
             2016-07-17                0               0.0   

                         incidents_28_day_prior_yr  incidents_7_day_prior_yr  \
pct crime    date                                                              
1   ABORTION 2016-06-19                          0                         0   
             2016-06-26                          0                         0   
             2016-07-03                          0                         0   
             2016-07-10                          0                         0   
             2016-07-17                          0                         0   

                        7_day_pct_change 28_day_pct_change  12_week_avg  \
pct crime    date                                                         
1   ABORTION 2016-06-19              *.*               *.*          0.0   
             2016-06-26              *.*               *.*          0.0   
             2016-07-03              *.*               *.*          0.0   
             2016-07-10              *.*               *.*          0.0   
             2016-07-17              *.*               *.*          0.0   

                         12_week_std  26_week_avg  26_week_std  52_week_avg  \
pct crime    date                                                             
1   ABORTION 2016-06-19          0.0          0.0          0.0          0.0   
             2016-06-26          0.0          0.0          0.0          0.0   
             2016-07-03          0.0          0.0          0.0          0.0   
             2016-07-10          0.0          0.0          0.0          0.0   
             2016-07-17          0.0          0.0          0.0          0.0   

                         52_week_std  same_28_day_avg  same_28_day_std  \
pct crime    date                                                        
1   ABORTION 2016-06-19          0.0              0.0              0.0   
             2016-06-26          0.0              0.0              0.0   
             2016-07-03          0.0              0.0              0.0   
             2016-07-10          0.0              0.0              0.0   
             2016-07-17          0.0              0.0              0.0   

                         four_conseq_inc  four_conseq_dec  incidents_ytd  \
pct crime    date                                                          
1   ABORTION 2016-06-19            False            False              0   
             2016-06-26            False            False              0   
             2016-07-03            False            False              0   
             2016-07-10            False            False              0   
             2016-07-17            False            False              0   

                         incidents_ytd_prior_yr ytd_pct_change  \
pct crime    date                                                
1   ABORTION 2016-06-19                       0            *.*   
             2016-06-26                       0            *.*   
             2016-07-03                       0            *.*   
             2016-07-10                       0            *.*   
             2016-07-17                       0            *.*   

                         12_week_pzscore  26_week_pzscore  52_week_pzscore  \
pct crime    date                                                            
1   ABORTION 2016-06-19              0.0              0.0              0.0   
             2016-06-26              0.0              0.0              0.0   
             2016-07-03              0.0              0.0              0.0   
             2016-07-10              0.0    

## Run Full Script

In [18]:
def main(data, ytd_counts=False):
    incidents_daily = create_daily_counts(crimes)
    trends = create_7_and_28_day_counts(incidents_daily)
    trends = create_rolling_avg_std(trends)
    trends = create_avg_std_of_same_28_day_period(trends)
    trends = create_four_conseq_inc_dec_flg(trends)
    trends = clean_up(trends)
    incidents_daily_strt = create_daily_counts_from_start_date(incidents_daily)
    if ytd_counts:
        trends = get_ytd_counts(trends)
    trends = calc_poisson_zscore(trends)
    trends = create_alert_score_and_classify(trends)
    return trends

trends = main(crimes)

trends.head()

incidents_7_day  incidents_28_day  \
pct crime    date                                            
1   ABORTION 2016-06-19                0               0.0   
             2016-06-26                0               0.0   
             2016-07-03                0               0.0   
             2016-07-10                0               0.0   
             2016-07-17                0               0.0   

                         incidents_28_day_prior_yr  incidents_7_day_prior_yr  \
pct crime    date                                                              
1   ABORTION 2016-06-19                          0                         0   
             2016-06-26                          0                         0   
             2016-07-03                          0                         0   
             2016-07-10                          0                         0   
             2016-07-17                          0                         0   

                        7_day_pct_change 28_day_pct_change  12_week_avg  \
pct crime    date                                                         
1   ABORTION 2016-06-19              *.*               *.*          0.0   
             2016-06-26              *.*               *.*          0.0   
             2016-07-03              *.*               *.*          0.0   
             2016-07-10              *.*               *.*          0.0   
             2016-07-17              *.*               *.*          0.0   

                         12_week_std  26_week_avg  26_week_std  52_week_avg  \
pct crime    date                                                             
1   ABORTION 2016-06-19          0.0          0.0          0.0          0.0   
             2016-06-26          0.0          0.0          0.0          0.0   
             2016-07-03          0.0          0.0          0.0          0.0   
             2016-07-10          0.0          0.0          0.0          0.0   
             2016-07-17          0.0          0.0          0.0          0.0   

                         52_week_std  same_28_day_avg  same_28_day_std  \
pct crime    date                                                        
1   ABORTION 2016-06-19          0.0              0.0              0.0   
             2016-06-26          0.0              0.0              0.0   
             2016-07-03          0.0              0.0              0.0   
             2016-07-10          0.0              0.0              0.0   
             2016-07-17          0.0              0.0              0.0   

                         four_conseq_inc  four_conseq_dec  12_week_pzscore  \
pct crime    date                                                            
1   ABORTION 2016-06-19            False            False              0.0   
             2016-06-26            False            False              0.0   
             2016-07-03            False            False              0.0   
             2016-07-10            False            False              0.0   
             2016-07-17            False            False              0.0   

                         26_week_pzscore  52_week_pzscore  \
pct crime    date                                           
1   ABORTION 2016-06-19              0.0              0.0   
             2016-06-26              0.0              0.0   
             2016-07-03              0.0              0.0   
             2016-07-10              0.0              0.0   
             2016-07-17              0.0              0.0   

                         same_28_day_pzscore  alert_mean trend_class  
pct crime    date                                                     
1   ABORTION 2016-06-19                  0.0         0.0      Normal  
             2016-06-26                  0.0         0.0      Normal  
             2016-07-03                  0.0         0.0      Normal  
             2016-07-10                  0.0         0.0      Normal  
             2016-07-17      

In [19]:
trends.to_csv('crime_trends_analysis.csv')